# Kernel PCA on ABA

In [14]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
%matplotlib inline

In [2]:
# Loading the main matrix
X = np.load('/data/bioprotean/ABA/MEMMAP/genes_list/ABA_mask_pos_std.npy')

In [ ]:
# PCA
pca = PCA(n_components = 10)
components = pca.fit_transform(10)

In [4]:
# Selecting 10 of the genes
X_10 = X[:,:10]
X_10.shape

(63113, 10)

In [ ]:
start_time = time.time()

# Running the KPCA
kpca = KernelPCA(n_components = 10, kernel = 'rbf')
components = kpca.fit_transform(X_10)

print('Running time is ', (time.time() - start_time))

In [ ]:
# Saving the components
np.save('/data/bioprotean/ABA/KernelPCA/rbf/KPCA_10_rbf.npy', components)

In [6]:
# Trying on a random array
rand_X = np.random.rand(3000,2000)

In [7]:
start_time = time.time()

# Running the KPCA
kpca = KernelPCA(n_components = 10, kernel = 'rbf')
components10 = kpca.fit_transform(rand_X)

print('Running time is ', (time.time() - start_time))

Running time is  4.071404695510864


In [34]:
start_time = time.time()

# Running the KPCA
kpca = KernelPCA(n_components = 100000, kernel = 'poly')
components20 = kpca.fit_transform(rand_X)

print('Running time is ', (time.time() - start_time))

Running time is  9.07273268699646


In [35]:
components20.shape

(3000, 3000)

In [25]:
start_time = time.time()

# Running the KPCA
kpca = PCA(n_components = 10)
components10 = kpca.fit_transform(rand_X)

print('Running time is ', (time.time() - start_time))

Running time is  0.4372990131378174


In [26]:
start_time = time.time()

# Running the KPCA
kpca = PCA(n_components = 20)
components20 = kpca.fit_transform(rand_X)

print('Running time is ', (time.time() - start_time))

Running time is  0.5185959339141846


In [20]:
np.array_equal(components10[:,0], components20[:,0])

False

In [27]:
components10[:,0] - components20[:,0]

array([ 0.17556096, -0.05270749,  0.11898546, ..., -0.78794115,
       -0.70759696,  0.75081371])

In [28]:
print(components10[:,0] , components20[:,0])

[ 0.22198653 -0.08887898  0.18252291 ... -0.56631888 -0.87766642
 -0.31449462] [ 0.04642556 -0.03617149  0.06353745 ...  0.22162227 -0.17006946
 -1.06530833]


In [2]:
#Visualizing one slice
c = components[:,0]
vol = c.reshape(67,58,41)
s = vol[20,:,:]
plt.imshow(s)

NameError: name 'np' is not defined

In [2]:
np.load()

NameError: name 'np' is not defined